# 0. 라이브러리 및 데이터

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
df = pd.read_csv('../1_Data/case_fraud_sent.csv', index_col=0)

In [3]:
with open("../2_Preprocessing/vocab_list/kkmaCorpus.pkl", "rb") as f:
    kkmaCorpus = pkl.load(f)

In [4]:
kkmaCorpus[0]

' 상고 이유 판단 불가 사후 행위 판단 유사 수신 행위 규제 법률 이하 유사 수신 행위법 제조 제항 제조 위반 행위 자체 사기 행위 해당 사기 행위 포함 유사 수신 행위법 위반죄 형법 제조 제항 사기죄 구성 요건 별개 범죄 서로 보호 법익 이상 유사 수신 행위 자가 출자자 별도 기망 행위 유사 수신 행위 조달 자금 전부 일부 투자 행위 유사 수신 행위법 위반죄 새 보호 법익 침해 유사 수신 행위법 위반죄 불가 사후 행위 별 죄인 사기죄 구성 원심 판시 이유 피고 인의 경 피해자 공소 외인 판시 사기 점 공소 사실 유사 수신 행위법 제조 금지 유사 수신 행위 기망 행위 포함 유사 수신 행위법 위반죄 사기죄 구성 요건 별개 범죄 서로 행위 태양 보호 법익 이유 기존 범죄인 유사 수신 행위법 위반죄 가벌 평가 범위 흡수 불가 사후 행위 해당 판단 원심 판결 이유 위 법리 기록 부분 원심 판단 일부 점 피고 인의 경 피해자 공소 외인 판시 사기죄 유사 수신 행위법 위반죄 불가 사후 행위 해당 원심 판단 불가 사후 행위 법리 오해 판결 영향 잘못 나머지 상고 이유 판단 부분 상고 이유 형사 소송법 제조 제호 형 가벼운 형 선고 사건 사실 오인 법리 오해 실질적 원심 증거 선택 증명력 판단 기초 사실 인정 원심 인정 사실 사실 관계 전제 법리 오해 지적 취지 주장 형 부당 취지 주장 해당 적법 상고 이유 결론 상고 기각 관여 대법관 일치 의견 주문 판결 대법관 민 유숙 재판장 이동 천대 엽주 심 영준'

# 1. 벡터화

In [15]:
# LDA 는 Count기반의 Vectorizer만 적용 
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(kkmaCorpus)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (2253, 1000)


# 2. 토픽모델링: LDA

In [ ]:
# 2. 토픽모델링: LDA
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 6개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=8, n_jobs=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

# 3. 토픽별 연관어 출력

In [ ]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index+1)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_feature_names_out( )를 통해 추출
feature_names = count_vectorizer.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

# 4. ⁣각 문서별로 가장 가까운 topic으로 할당

In [16]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌
doc_topic = lda.transform(feat_vect)
doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])
display(doc_topic_df.head())

# 실제 데이터와 조인 
doc_topic_df = doc_topic_df.join(df)
doc_topic_df.tail()

,Doc_Num,Topic,Percentage
0,0,4,0.703028
1,1,2,0.967383
2,2,2,0.996565
3,3,2,0.999755
4,4,4,0.681904


,Doc_Num,Topic,Percentage,판례일련번호,판례내용_전처리2
2248,2248,7,0.720403,215295,피고인의 상고이유는 단기 년 월 일에 피고인은 목포지검 김대운 검사님으로 부터 ...
2249,2249,7,0.442109,215359,대구고등검찰청 검사장 대리검사 주운화의 상고취의 제은 공소사실 중 제 사기사실에...
2250,2250,7,0.843715,86037,대구고등검찰청 검사장 대리검사 이호용의 상고이유는 원판결은 기이유로서 피고인 ...
2251,2251,7,0.489123,86031,대구고등검찰청 검사의 상고취의는 제.공소 사실의 요지는 피고인은 서기 년 월 중...
2252,2252,7,0.981761,188582,서울고등검찰청 검사 권오병 상고이유는 원심은 소송절차가 적법히 시행되었다고 볼 ...


# 5. 토픽별 문서 수 계산

In [17]:
display(doc_topic_df.groupby('Topic')[['Doc_Num']].count())

,Doc_Num
Topic,
0,156
1,226
2,151
3,129
4,214
5,353
6,92
7,932


# 6. LDA 시각화: pyLDAvis

In [18]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis) 

In [14]:
pyLDAvis.save_html(vis, 'LDA_image.html')